<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



file1 = open(r"C:\tmp\BERT_results\zazu.txt", 'w')
L = ["This is Delhi\n", "This is Paris\n", "This is London\n"]
s = "Hello\n"
file1.write(s)
file1.writelines(L)
file1.close()
file1 = open(r"C:\tmp\BERT_results\zazu.txt", 'r')
print(file1.read())
file1.close()



!pwd
!cd /..
!pwd
!ls -la /content/ML_GitHub/BERT_results/checkpoint-80
!cat /content/ML_GitHub/BERT_results/checkpoint-80/trainer_state.json
from google.colab import files
files.download(r"/content/ML_GitHub/BERT_results/checkpoint-80/trainer_state.json")





In [1]:
from getpass import getpass
HF_TOKEN = getpass("Enter your Hugging Face token: ")

Enter your Hugging Face token: ··········


In [2]:
from huggingface_hub import login
login(token=HF_TOKEN, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import os, sys, shutil

os.chdir("/content")
current_dir = os.getcwd()
print(f"The current directory is {current_dir}")

if os.path.isdir('ML_GitHub'):
    shutil.rmtree('ML_GitHub')

!git init
!git branch -m main
!git clone https://github.com/claudelepere/ML_GitHub.git
print(f"list /content: {os.listdir(current_dir)}")
print(f"list /content/ML_GitHub: {os.listdir(current_dir + '/ML_GitHub')}")
!ls -la /content/ML_GitHub/datasetHF
os.chdir(current_dir + '/ML_GitHub')

#!pip install fsspec==2024.10.0
!pip install -q transformers datasets
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk('datasetHF')    # from disk does not mean from local disk
print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")

The current directory is /content
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Cloning into 'ML_GitHub'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 95 (delta 48), reused 56 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 10.03 MiB | 8.94 MiB/s, done.
Resolving deltas: 100% (48/48), done.
list /content: ['.config', '.git', 'ML_GitHub', 'sample_data']
list /content/ML_GitHub: ['.gi

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's check the first example of the training split:

In [4]:
example = dataset['validation'][0]
#example

The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [5]:
labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()

print(len(labels), list(enumerate(labels)))
id2label = {idx:label for idx, label in enumerate(labels)}
print(id2label)
label2id = {label:idx for idx, label in enumerate(labels)}
print(len(labels), labels)

42 [(0, '142'), (1, '146'), (2, '147'), (3, '148'), (4, '149'), (5, '150'), (6, '151'), (7, '152'), (8, '153'), (9, '154'), (10, '155'), (11, '156'), (12, '157'), (13, '158'), (14, '160'), (15, '162'), (16, '165'), (17, '167'), (18, '168'), (19, '169'), (20, '170'), (21, '171'), (22, '173'), (23, '174'), (24, '176'), (25, '356'), (26, '360'), (27, '361'), (28, '362'), (29, '364'), (30, '375'), (31, '376'), (32, '394'), (33, '408'), (34, '409'), (35, '685'), (36, '686'), (37, '689'), (38, '756'), (39, '758'), (40, '760'), (41, '761')]
{0: '142', 1: '146', 2: '147', 3: '148', 4: '149', 5: '150', 6: '151', 7: '152', 8: '153', 9: '154', 10: '155', 11: '156', 12: '157', 13: '158', 14: '160', 15: '162', 16: '165', 17: '167', 18: '168', 19: '169', 20: '170', 21: '171', 22: '173', 23: '174', 24: '176', 25: '356', 26: '360', 27: '361', 28: '362', 29: '364', 30: '375', 31: '376', 32: '394', 33: '408', 34: '409', 35: '685', 36: '686', 37: '689', 38: '756', 39: '758', 40: '760', 41: '761'}
42 ['14

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [6]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# examples and not example because batched=True => examples is a batch
def preprocess_data(examples, indices):
  # take a batch of texts
  text = examples["text"]
  #print("Indices:", indices, "Batch size:", len(text), "Num labels:", len(labels))

  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)

  labels_batch = {label: np.zeros(len(text)) for label in labels}                                                # validation: 42 rows: 'label': 18 x 0.0
  #print(f"labels_batch: zeros: {type(labels_batch)} {len(labels_batch)} {labels_batch}")
  #print(f"examples.keys(): {examples.keys()}")
  # examples.keys(): all the column names (keys) in the batch dict "examples"
  # for each k in labels, a new key-value pair k: examples[k] is added to labels_batch
  labels_batch.update({k: [1.0 if val else 0.0 for val in examples[k]] for k in examples.keys() if k in labels}) # validation: 42 rows: 'label': 1.0 or 0.0

  #print(f"labels_batch: updated: {type(labels_batch)} {len(labels_batch)} {labels_batch}")

  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))                                                             # validation: 18 rows 42 columns
  #print(f"labels_matrix:{type(labels_matrix)} {labels_matrix.shape}")
  # fill numpy array
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")                                                                            # labels are sorted
    labels_matrix[:, idx] = labels_batch[label] # for each row, sets the idx-th column of labels_matrix with the values from labels_batch[label]

  #print("First row of labels_matrix:", labels_matrix[0])

  # Add labels to encoding
  encoding["labels"] = labels_matrix.tolist()
  print(f"encoding['labels']: {encoding['labels']}")


  return encoding



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names, with_indices=True)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

encoding['labels']: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [8]:
example = encoded_dataset['validation'][0]
print(f"example.keys(): {example.keys()}")
print(f"example['input_ids']: {example['input_ids']}")
print(f"example['token_type_ids']: {example['token_type_ids']}")
print(f"example['attention_mask']: {example['attention_mask']}")
print(f"example['labels']: {example['labels']}")

example.keys(): dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
example['input_ids']: [101, 2552, 15735, 2015, 1011, 8040, 6824, 3040, 1013, 16216, 16643, 2239, 20589, 2139, 4013, 15759, 2015, 8040, 6824, 3040, 1010, 10026, 2139, 4013, 15759, 2015, 1010, 1012, 5658, 2552, 15735, 2015, 3393, 2326, 4503, 5051, 3672, 1040, 1005, 2552, 15735, 2015, 9765, 17202, 1040, 1005, 16655, 7990, 18175, 2139, 2711, 5267, 21864, 19817, 12462, 10484, 3372, 29032, 9515, 6216, 3672, 7505, 4078, 4013, 15759, 2015, 25272, 2595, 1006, 9998, 20236, 4310, 1010, 5127, 14192, 2063, 10364, 2474, 16216, 16643, 2239, 4078, 4262, 4372, 7913, 2552, 15735, 2015, 3802, 4649, 2112, 8189, 7442, 2015, 1010, 5127, 14192, 2063, 10364, 2474, 16216, 16643, 2239, 4078, 4262, 4372, 7913, 2552, 15735, 2015, 3802, 4649, 12666, 26744, 1010, 28616, 2063, 4372, 2173, 1040, 1005, 4895, 25272, 13675, 2213, 1010, 4385, 1012, 1007, 1012, 29536, 7913, 2535, 4372, 9092, 2102, 10861, 8040, 6824, 3040, 1013, 16216, 1

In [9]:
tokenizer.decode(example['input_ids'])

"[CLS] actiris - scrum master / gestionnaire de projets scrum master, chef de projets,. net actiris le service developpement d'actiris est compose d'une trentaine de personnes qui travaillent essentiellement sur des projets nouveaux ( dossier unique, plateforme pour la gestion des relations entre actiris et les partenaires, plateforme pour la gestion des relations entre actiris et les employeurs, mise en place d'un nouveau crm, etc. ). votre role en tant que scrum master / gestionnaire de projets, vous initiez et coordonnez, dans un environnement agile, des projets dans le cadre du developpement de nos nouvelles plateformes core business et vous veillez a leur bonne realisation dans les delais impartis. vos responsabilites vous collectez les informations necessaires au bon deroulement du projet et etes capable de discuter d'un backlog avec le metier ; vous determinez les etapes cles, planning et retro - planning highlevel sans forcement de requete aupres de l'equipe de developpement ; 

In [10]:
example = encoded_dataset['validation'][0]
print(f"example['labels']: {example['labels']}")


example['labels']: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]


['169', '170', '375']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [12]:
encoded_dataset.set_format("torch")
example = encoded_dataset['validation'][0]
print(f"example['labels']:  {type(example['labels'])} {example['labels'].shape}\n{example['labels']}")

#raise Exception("STOP")

example['labels']:  <class 'torch.Tensor'> torch.Size([42])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])


## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#raise Exception("STOP")

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [15]:
batch_size = 8
metric_name = "f1"

In [16]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = r'C:\tmp\BERT_results\output',
    overwrite_output_dir=True,
    logging_dir= r'C:\tmp\BERT_results\logs',
    logging_steps=50,
    save_steps=100,
    save_total_limit=2,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [17]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score, average_precision_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.2):
    _average = 'micro'    # 'micro' or 'weighted'
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true               = labels

    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=_average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=_average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {'f1'                  : f1,
               'precision'           : precision,
               'recall'              : recall,
               'roc_auc'             : roc_auc,
               'precision_recall_auc': precision_recall_auc,
               'accuracy'            : accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [18]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [19]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1999,  3388,  2819,  1011,  2613,  3527, 13728,  2368,  1011,
        24296,  6112,  3992, 24296,  6112,  1999,  3388,  2819,  1011,  2613,
         3527, 13728,  2368,  2115,  3853,  2000,  2490,  1996, 27258,  3930,
         1997,  2256, 24296,  3218,  1010,  2057,  2024,  2559,  2005,  2195,
        24296,  6112,  6145,  1012,  2182,  1005,  1055,  2129,  2017,  1005,
         2222,  2191,  4254,  1024,  2005,  3469,  3934,  2017,  2147,  1999,
         2136,  2007,  2060, 24296,  6112,  6145,  1010,  6112,  5576,  8160,
         1998,  2622, 10489,  2000,  4339,  1037,  2047,  3112,  2466,  1012,
         2057,  2064,  4175,  2006,  2017,  2005,  1996,  2658,  7375,  1997,
         1996,  8518, 18011,  2000,  2017,  1012,  2005,  3760,  6112,  3934,
         2017,  2024,  1999,  3715,  2005,  1996,  2440,  8147,  1024,  2017,
        18012,  2115,  7396,  2013,  1996,  2640,  2127,  2010,  5576,  2003,
         3929,  6515,  1012,  1999,  2804,  2000, 24853,  1010, 

In [20]:
#forward pass
print(f"inputids: {type(encoded_dataset['train']['input_ids'][0])} {encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask: {type(encoded_dataset['train']['attention_mask'][0])} {encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels: {type(encoded_dataset['train'][0]['labels'])} {encoded_dataset['train'][0]['labels'].shape}")

outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0),
                attention_mask=encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
                labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

inputids: <class 'torch.Tensor'> torch.Size([512])
attention_mask: <class 'torch.Tensor'> torch.Size([512])
labels: <class 'torch.Tensor'> torch.Size([42])


SequenceClassifierOutput(loss=tensor(0.7126, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.2585, -0.1224, -0.4430, -0.4935, -0.0236, -0.2207,  0.0305,  0.1147,
          0.3070,  0.7661, -0.9686, -0.1501, -0.2956, -0.2866,  0.0652,  0.1075,
         -0.3061,  0.0324,  0.6226, -0.0025,  0.1019, -0.6912, -0.4262,  0.1098,
         -0.3319, -0.5422, -0.0437,  0.4820,  0.5397, -0.7148,  0.4940, -0.1285,
          0.8474, -0.0568,  0.3942, -0.3448,  0.3767,  0.1390,  0.4895,  0.1119,
         -0.1934,  0.4585]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,No log,0.596951,0.102886,0.054233,1.000000,0.522088,0.082597,0.000000
2,No log,0.527805,0.102886,0.054233,1.000000,0.512639,0.095246,0.000000
3,No log,0.488855,0.102886,0.054233,1.000000,0.508340,0.068972,0.000000


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,No log,0.596951,0.102886,0.054233,1.000000,0.522088,0.082597,0.000000
2,No log,0.527805,0.102886,0.054233,1.000000,0.512639,0.095246,0.000000
3,No log,0.488855,0.102886,0.054233,1.000000,0.508340,0.068972,0.000000
4,0.568500,0.469148,0.103015,0.054305,1.000000,0.507454,0.078310,0.000000
5,0.568500,0.462860,0.100629,0.053050,0.975610,0.518199,0.081562,0.000000


TrainOutput(global_step=80, training_loss=0.5328575253486634, metrics={'train_runtime': 5246.2254, 'train_samples_per_second': 0.122, 'train_steps_per_second': 0.015, 'total_flos': 168451552051200.0, 'train_loss': 0.5328575253486634, 'epoch': 5.0})

In [23]:
trainer.save_model("my_model")    # Save the trained model and tokenizer: saves the model weights, the tokenizer, the model configuration file ("config.json")

import json

with open("training_metrics.json", 'w') as f:
    json.dump(trainer.state.log_history,f)


## Evaluate

After training, we evaluate our model on the validation set.

In [24]:
eval_results = trainer.evaluate()

In [25]:
with open("eval_metrics.json", 'w') as f:
    json.dump(eval_results, f)

In [30]:
from google.colab import drive
drive.mount('/content/drive')

# Copy files to your Google Drive
!cp -r my_model /content/drive/MyDrive/
!cp training_metrics.json /content/drive/MyDrive/
!cp eval_metrics.json /content/drive/MyDrive/



Mounted at /content/drive


In [26]:
from google.colab import files

!zip -r -9 my_model.zip my_model
!split -b 100M my_model.zip my_model_part_

# for part in ['my_model_part_aa', 'my_model_part_ab', 'my_model_part_ac']:  # Adjust based on number of parts
#    files.download(part)
files.download("my_model.zip")
files.download("training_metrics.json")
files.download("eval_metrics.json")

#raise Exception("STOP")

  adding: my_model/ (stored 0%)
  adding: my_model/tokenizer.json (deflated 71%)
  adding: my_model/training_args.bin (deflated 51%)
  adding: my_model/tokenizer_config.json (deflated 76%)
  adding: my_model/special_tokens_map.json (deflated 42%)
  adding: my_model/config.json (deflated 62%)
  adding: my_model/model.safetensors (deflated 7%)
  adding: my_model/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Inference

Let's test the model on a new sentence:

id: 323697
"Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten,"

['142', '147', '149', '154', '156', '157', '173', '409', '685', '689']

---

id: 323611,"Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home","171,170,794,800,798,797,138,139,352"
---


In [37]:
text = "Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten"
#text = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [38]:
logits = outputs.logits
logits.shape

torch.Size([1, 42])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [39]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

[]


**id**: 323697

**MySQL**: "142,189,190,754,208,794,676,811,812,139,138" (only 142="Developer / Analyst Programmer") is a 7-skill)

**predicted_labels**: ['148', '152', '154', '409'] : all are 7-skills: 148="Technical Analyst", 152="Technical Writer", 154="Database Admininistrator"


---


**id**: 323611

**MySQL**: "171,170,794,800,798,797,138,139,352"            
           171: Project Mgmt Officer (PMO)  
           170: Project Manager / Coordinator

**predicted labels**: 409:  
                      409: "SOA Specialist" (SOA: Service Oriented Architecture)